# Feature 
1)Identify the reference location in the feature file for +/- strand 

2)Extract "CG"-center pattern rows from the original feature file

In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


## Load the CG test data

In [5]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split, StratifiedKFold

import tensorflow as tf
import tensorflow_io as tfio

from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, Sequential
print("TensorFlow version:", tf.__version__)
print("tensorflow_io:", tfio.__version__)

2021-11-04 12:50:48.858228: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/local/apps/gcc/9.2.0/lib:/cm/local/apps/gcc/9.2.0/lib64:/cm/shared/apps/slurm/current/lib64/slurm:/cm/shared/apps/slurm/current/lib64
2021-11-04 12:50:48.858271: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


TensorFlow version: 2.6.1
tensorflow_io: 0.21.0


In [6]:
input_path='/pod/2/li-lab/Ziwei/Nanopore/daily/test/'

### Name the feature
names_feature = ['chrom', 'site_pos', 'align_strand', 'pos_in_strand', 'read_id', 'read_strand',
                 'kmer_seq', 'kmer_signal_mean', 'kmer_signal_std', 'kmer_signal_length', 'kmer_signal_range',
                 'label']

In [2]:
####site_pos: start position in 0-based,
##if alignd_strand = +, the pos is the C or G location in the reference, pos = [site_pos, site_pos + 1]
##if alignd_strand = -, the pos is the G or C location in the reference (C in the kmer still), pos = [site_pos, site_pos + 1]
##Filter is in need to pick up the CG pattern
df=pd.read_csv(os.path.join(input_path,'feature.CG.test.csv'),sep='\t', names = names_feature)
df.head(3)

,chrom,site_pos,align_strand,pos_in_strand,read_id,read_strand,kmer_seq,kmer_signal_mean,kmer_signal_std,kmer_signal_length,kmer_signal_range,label
0,J02459.1,42105,-,6396,17ff58dc-512e-4b3a-abb8-5ab1bdb42f6a,t,TGATACTTCGTCGCTGT,"0.643744,-1.4208,-1.009653,1.055123,-0.112415,...","0.187366,0.141536,0.159047,0.5128,0.257095,0.1...","13,28,6,38,18,21,3,7,17,21,14,14,10,20,15,11,16","0.736942,0.549584,0.44966,2.473129,1.13664,0.5...",0
1,J02459.1,42102,-,6399,17ff58dc-512e-4b3a-abb8-5ab1bdb42f6a,t,TACTTCGTCGCTGTCTC,"1.055123,-0.112415,-0.446686,0.724452,0.999244...","0.5128,0.257095,0.15307,0.250642,0.107862,0.19...","38,18,21,3,7,17,21,14,14,10,20,15,11,16,8,14,6","2.473129,1.13664,0.549585,0.574565,0.362226,0....",0
2,J02459.1,42094,-,6407,17ff58dc-512e-4b3a-abb8-5ab1bdb42f6a,t,CGCTGTCTCGCCACACG,"0.192711,-0.661999,0.319133,1.106663,-0.936791...","0.124343,0.115293,0.155229,0.127607,0.292445,0...","14,10,20,15,11,16,8,14,6,5,24,7,32,7,8,3,7","0.412188,0.399697,0.587056,0.437169,1.086678,1...",0


In [3]:
df['label'].value_counts()

1    1200
0    1000
2     800
Name: label, dtype: int64

### Define load the data

In [64]:
### Define load the data
names_feature = ['chrom', 'site_pos', 'align_strand', 'pos_in_strand', 'read_id', 'read_strand',
                 'kmer_seq', 'kmer_signal_mean', 'kmer_signal_std', 'kmer_signal_length', 'kmer_signal_range',
                 'label']

def load_df(input_path):
    df = pd.read_csv(os.path.join(input_path),sep='\t',  names = names_feature)
    
    ## Drop last column of a dataframe
    df_feature = df.iloc[: , :-1]
    df_class = df[df.columns[-1]]
    df_class = np.squeeze(df_class) #Convert the label into 1d-array
        
    assert df_feature.shape[0] == df_class.shape[0]
    return df_feature, df_class

In [65]:
input_path='/pod/2/li-lab/Ziwei/Nanopore/daily/test/feature.CG.test.csv'
df_feature, df_class = load_df(input_path)
df_feature.shape, df_class.shape

((3000, 11), (3000,))

In [66]:
df_feature.head(3)

,chrom,site_pos,align_strand,pos_in_strand,read_id,read_strand,kmer_seq,kmer_signal_mean,kmer_signal_std,kmer_signal_length,kmer_signal_range
0,J02459.1,42105,-,6396,17ff58dc-512e-4b3a-abb8-5ab1bdb42f6a,t,TGATACTTCGTCGCTGT,"0.643744,-1.4208,-1.009653,1.055123,-0.112415,...","0.187366,0.141536,0.159047,0.5128,0.257095,0.1...","13,28,6,38,18,21,3,7,17,21,14,14,10,20,15,11,16","0.736942,0.549584,0.44966,2.473129,1.13664,0.5..."
1,J02459.1,42102,-,6399,17ff58dc-512e-4b3a-abb8-5ab1bdb42f6a,t,TACTTCGTCGCTGTCTC,"1.055123,-0.112415,-0.446686,0.724452,0.999244...","0.5128,0.257095,0.15307,0.250642,0.107862,0.19...","38,18,21,3,7,17,21,14,14,10,20,15,11,16,8,14,6","2.473129,1.13664,0.549585,0.574565,0.362226,0...."
2,J02459.1,42094,-,6407,17ff58dc-512e-4b3a-abb8-5ab1bdb42f6a,t,CGCTGTCTCGCCACACG,"0.192711,-0.661999,0.319133,1.106663,-0.936791...","0.124343,0.115293,0.155229,0.127607,0.292445,0...","14,10,20,15,11,16,8,14,6,5,24,7,32,7,8,3,7","0.412188,0.399697,0.587056,0.437169,1.086678,1..."


In [63]:
df_class.head(3)

0    0
1    0
2    0
Name: label, dtype: int64

## Train_split into cross-validation and testing dataset

2021/11/04

Right now I didn't use cross validation (Need Yue's help in the future)

Instead, I am using train | valid | test dataset (8:1:1)

Reference: https://towardsdatascience.com/how-to-split-data-into-three-sets-train-validation-and-test-and-why-e50d22d3e54c

In [91]:
def train_val_test(df):
    # split the data in training and remaining dataset
    X_train, X_rem, y_train, y_rem = train_test_split(X,
                                                      y, 
                                                      train_size=0.8,
                                                      stratify=y,
                                                      random_state=42)

    # Now since we want the valid and test size to be equal (10% each of overall data). 
    # we have to define valid_size=0.5 (that is 50% of remaining data)
    X_valid, X_test, y_valid, y_test = train_test_split(X_rem,
                                                        y_rem, 
                                                        test_size=0.5,
                                                        stratify=y_rem,
                                                        random_state=13)
    
    ## Reset the index
    X_train = X_train.reset_index(drop=True)
    X_valid = X_valid.reset_index(drop=True)
    X_test = X_test.reset_index(drop=True)
    
#    print(X_train.shape), print(y_train.shape)
#    print(X_valid.shape), print(y_valid.shape)
#    print(X_test.shape), print(y_test.shape)
    return X_train, X_valid, X_test, y_train, y_valid, y_test

In [89]:
X_train, X_valid, X_test, y_train, y_valid, y_test = train_val_test(df)
print(X_train.shape), print(y_train.shape)
print(X_valid.shape), print(y_valid.shape)
print(X_test.shape), print(y_test.shape)

(2400, 11)
(2400,)
(300, 11)
(300,)
(300, 11)
(300,)


(None, None)

In [90]:
X_train.head(3)

,chrom,site_pos,align_strand,pos_in_strand,read_id,read_strand,kmer_seq,kmer_signal_mean,kmer_signal_std,kmer_signal_length,kmer_signal_range
0,J02459.1,33840,+,33840,dea70d60-bb7f-4227-aaf8-f08ebf58ac31,t,GTCATGGTCGATTTCAA,"-1.203785,1.389703,0.059888,-0.363606,0.795389...","0.131342,0.259303,0.352943,0.664445,0.272477,0...","22,10,17,14,12,3,22,7,3,4,3,25,3,21,11,6,16","0.661764,0.967192,1.628956,2.532518,1.069002,0..."
1,J02459.1,29679,+,29679,b63c008a-57a8-4e28-93a2-0f870e8fe70e,t,ATGTATGTCGTTTCAGC,"-1.116067,0.75724,-0.256056,0.56832,-0.735381,...","0.122976,0.328393,0.446919,0.194609,0.114892,0...","17,8,4,4,8,9,5,5,6,5,7,8,6,15,7,15,6","0.524603,1.099169,1.111659,0.499623,0.387207,1..."
2,KJ477685.1,36602,-,130213,d9ef0594-86f0-43d4-8b7f-69a41735a903,t,AGGAGCTTCGGCCCTTT,"0.431151,-0.567246,-1.255237,-1.063179,-0.9412...","0.145854,0.120132,0.1851,0.082702,0.123245,0.0...","7,21,5,6,3,11,3,9,12,19,3,3,5,45,20,3,37","0.422985,0.548737,0.537305,0.217209,0.297233,0..."


In [92]:
type(y_train)

numpy.ndarray

# Process the data

## Repeat the code from DeepMP

In [93]:
####Repeat the code from DeepMP
#https://github.com/pepebonet/DeepMP/blob/3c2dcdc3c1d6f0c3dc6ab087e30de74e3a81c257/deepmp/utils.py
base2code_dna = {'A': 0, 'C': 1, 'G': 2, 'T': 3, 'N': 4}

def kmer2code(kmer_bytes):
    return [base2code_dna[x] for x in kmer_bytes]

### Extract features as numpy.ndarray
chrom = df['chrom'].values
site_pos = df['site_pos'].values
align_strand = df['align_strand'].values
pos_in_strand = df['pos_in_strand'].values
read_id = df['read_id'].values
read_strand = df['read_strand'].values
label = df['label'].values

kmer_seq = df['kmer_seq'].apply(kmer2code)


### Extract features as list
#https://www.tensorflow.org/api_docs/python/tf/strings/to_number
#Converts each string in the input Tensor to the specified numeric type.
kmer_mean = [tf.strings.to_number(i.split(',')) for i in df['kmer_signal_mean']]
kmer_std = [tf.strings.to_number(i.split(',')) for i in df['kmer_signal_std']]
kmer_length = [tf.strings.to_number(i.split(',')) for i in df['kmer_signal_length']]
kmer_range = [tf.strings.to_number(i.split(',')) for i in df['kmer_signal_range']]

In [94]:
import h5py
file_name = '/pod/2/li-lab/Ziwei/Nanopore/daily/2021-11-03/test.h5'

#with h5py.File(file_name, 'a') as hf:
#    hf.create_dataset("kmer",  data=np.stack(kmer_seq), chunks=True, maxshape=(None,None))
#    hf.create_dataset("signal_mean",  data=np.stack(kmer_mean), chunks=True, maxshape=(None,None))
#    hf.create_dataset("signal_std",  data=np.stack(kmer_std), chunks=True, maxshape=(None,None))
#    hf.create_dataset('label',  data=label, chunks=True, maxshape=(None,))
#    hf.create_dataset('chrom',  data=chrom, chunks=True, maxshape=(None,))
#    hf.create_dataset('strand',  data=align_strand, chunks=True, maxshape=(None,))

with h5py.File(file_name, 'r') as hf:
    bases = hf['kmer'][:]
    signal_mean = hf['signal_mean'][:]
    signal_std = hf['signal_std'][:]

In [95]:
signal_mean.shape, signal_std.shape

((3000, 17), (3000, 17))

In [96]:
type(label)

numpy.ndarray

In [624]:
test = tf.reshape(signal_mean, [-1, kmer, 1])
test.dtype

tf.float32

In [625]:
embedding_size = 4
embedded_bases = tf.one_hot(bases, embedding_size)
embedded_bases.shape, type(signal_mean), embedded_bases.dtype

(TensorShape([3000, 17, 4]), numpy.ndarray, tf.float32)

In [187]:
signal_mean

array([[ 0.643744, -1.4208  , -1.009653, ...,  1.106663, -0.936791,
         1.488718],
       [ 1.055123, -0.112415, -0.446686, ..., -0.126467,  1.381098,
         0.143641],
       [ 0.192711, -0.661999,  0.319133, ...,  0.134273, -0.020817,
        -1.466747],
       ...,
       [-1.200431, -1.228535, -0.590179, ..., -0.887275, -1.051214,
        -0.630327],
       [ 0.987012, -0.259816, -0.887275, ...,  0.44966 ,  0.772971,
         0.810994],
       [-0.67449 , -0.875231,  1.48663 , ..., -0.0542  ,  1.335931,
        -0.477764]], dtype=float32)

## implentation for my own tensor process

Write a function to prepare all the features for tensor

In [45]:
###Follow the tensorflow encode way: https://www.tensorflow.org/io/tutorials/genome?hl=hr
###Potential issue: I didn't consider other bases here, tf exception is for [1,1,1,1]
###The one hot encoding is following the alphabetical order

base_num = {'A': 0, 'C':1, 'G':2, 'T': 3}
def kmer2num(kmer_seq):
    return [base_num[base] for base in kmer_seq]

### Prepare for the tensor
def array2tensor(input_array):
    tensor_2D = tf.convert_to_tensor(input_array, dtype=float) ##Convert 2D-array into tensor
    tensor_3D = tf.expand_dims(tensor_2D, -1) #-1 denotes the last dimension
#    tensor_3D = tf.reshape(tensor_2D, shape=[tensor_2D.shape[0], tensor_2D.shape[1], 1])
    return tensor_3D

def process_tensor(df_feature, df_label, kmer=17):
    ### Extract features as numpy.ndarray
#    chrom = df['chrom'].values
#    site_pos = df['site_pos'].values
#    align_strand = df['align_strand'].values
#    pos_in_strand = df['pos_in_strand'].values
#    read_id = df['read_id'].values
#    read_strand = df['read_strand'].values
    label = df_label.values
    
    kmer_mean = df['kmer_signal_mean'].str.split(",", n = kmer, expand = True).to_numpy(dtype=float)
    kmer_std = df['kmer_signal_std'].str.split(",", n = kmer, expand = True).to_numpy(dtype=float)
    kmer_length = df['kmer_signal_length'].str.split(",", n = kmer, expand = True).to_numpy(dtype=float)
    kmer_range = df['kmer_signal_range'].str.split(",", n = kmer, expand = True).to_numpy(dtype=float)

    kmer_seq = np.stack(df['kmer_seq'].apply(kmer2num))
    
    ## Prepare the concated tensors
    ###Prepare tensors for model
    kmer_seq_tensor = tf.one_hot(kmer_seq, 4) ###4 bases
    
    ### Concate the tensor
    tensor = tf.concat([kmer_seq_tensor,
                        array2tensor(kmer_mean),
                        array2tensor(kmer_std),
                        array2tensor(kmer_length),
                        array2tensor(kmer_range)],
                        axis = 2) ##concate the 3rd dimension
    
    ###Make sure the feature/label has the same dimension
    assert tensor.shape[0] == label.shape[0]
    
    return tensor, label

In [46]:
tensor, label = process_tensor(df, kmer=17)
tensor.shape, label.shape

(TensorShape([3000, 17, 8]), (3000,))

## Train_test_Split

((3000, 11), (3000,))

((2400, 11), (600, 11))

TensorFlow version: 2.6.1
tensorflow_io: 0.21.0
